# 为什么要写 typehint

## 提升代码可读性

想让代码可读性更好？又不想写太多注释或者 docstring？那么 typehint 就是你的不二之选。

In [ ]:
import numpy as np
def add(a, b):
    return a + b

In [ ]:
import numpy as np

def add(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return a + b

c = add(np.array(1), np.array(2))

## 利用 IDE，提升编程体验

如果你经常使用 PyTorch 的接口，例如：

In [ ]:
import torch

x = torch.Tensor(1)


# 1. x. 可以直接出现可用接口提示

# 2. tensor 进行一些运算后，仍然有类型提示

x = x.view(-1, 1)

## 利用 typehints 开发实用特性

### MMEval 基于 TypeHints 实现分发

```python
# Code in MMEval

class Accuracy(BaseMetric):
    @overload  # type: ignore
    @dispatch
    def _compute_corrects(
        self, 
        predictions: Union['torch.Tensor', Sequence['torch.Tensor']],
        labels: Union['torch.Tensor', Sequence['torch.Tensor']]) -> 'torch.Tensor':
        ...
    
    @overload  # type: ignore
    @dispatch
    def _compute_corrects(
        self, 
        predictions: Union['tensorflow.Tensor', Sequence['tensorflow.Tensor']],
        labels: Union['tensorflow.Tensor', Sequence['tensorflow.Tensor']]) -> 'tensorflow.Tensor':
```

### PyTorch 基于 TypeHints 实现 torch.jit.script

![img_v2_72963cb6-bdba-4338-ab54-a364af948c1g](https://user-images.githubusercontent.com/57566630/204156355-d0ca1683-bcae-4bd9-a3a6-1dcf6c266d9f.jpg)


## 增加代码鲁棒性

见[鲁棒性示例](./typehints/robustness.py)

# 怎么写 TypeHints

## 基本类型

In [ ]:
# 声明类型 + 定义 
a: int = 1
# 先声明 
b: int
# 后定义 
b = 1 
# str 类型的 Type Hints 
repo: str = 'mmcv' 
# float 类型的 Type Hints  
value: float = 0.1 
# 函数中的 typehint, 输出类型用 -> 连接 
def foo(a: int, b: int=1) -> int: 
    return a + b 

## 复合型的 Type Hints

In [ ]:
from typing import Any
info: dict = dict(a=1, b=2)  # 等价于 Dict[Any, Any] 
element: list = [1]  # 等价于 List[Any] 

上述写法仅声明了变量本身的类型是 list/dict，而变量中的元素可以是任意类型（Any）。如果想进一步约束容器中元素的类型，则需要引入 typing 模块的 Dict 和 List：

In [ ]:
from typing import List, Tuple, Dict 
int_list: List[int] = [1, 2] 
int_tuple: Tuple[int, int] = (1, 1) 
str_int_dict: Dict[str, int] = dict(name=1) 

如果变量可能是多种类型，则需要引入 Union：

In [ ]:
from typing import Union
int_or_float: Union[int, float] = 1 
int_or_float = 0.5

Optional 类来声明默认值为 None 的实例：

In [ ]:
from typing import Optional

# 除了 None 以外只有可能是一种类型的 typehint 用 Optional
single_none: Optional[str] = None 

# 除 None 以外，可能是多种类型的 typehint 用 Union
multi_none: Union[str, int, None] = None
# Optional[Union[str, int]]

复杂一点的例子：

In [ ]:
complex_type = Union[List[int], int, None]

## Type Hints 别名

有些变量类型的 Type Hints 过于复杂，直接在函数中声明会影响接口的可读性，因此可以重命名特定的变量类型：

In [ ]:
complex_type = Union[List[int], int, None]
 
def foo(comlex_arg: complex_type = None) -> None: 
    pass 

## 函数类型的 Type Hints

可以使用 Callable 来声明函数类型的变量。直接使用 Callable 表示函数接受任意个数、任意类型的参数，并返回任意个数、任意类型的变量。如果想进一步约束函数入参和返回值类型，可以使用 Callable[[Arg1Type, Arg2Type], ReturnType]：

In [ ]:
from typing import Callable 
 
def foo(a: int, b: int) -> int: 
    return a + b 
 
# 不声明参数和返回值类型 
def register_callback2(func: Callable):
    pass 
 
# 声明入参类型和返回类型。入参为 （int, int），返回值类型为 int 
def register_callback1(func: Callable[[int, int], int]): 
    pass 

## Type Hints 对应的 docstring


参数有默认值。直接在对应的 args 末尾加 Defaults to xxx 即可：

In [ ]:
def hello(name: str = 'heihei') -> None: 
 """Say hello to someone.
 
    Args: 
    name (str): name of people. Defaults to "heihei". 
    """ 

參數有 None 作为默认值，则在括号中额外申明 optional

In [ ]:
def hello(name: Optional[str] = None) -> None: 
 """Say hello to someone. 
 
    Args: 
    name (str, optional): name of people. Defaults to None
    """ 

参数可能是多种类型。可以写作 （str or List[str]）或者 （str | List[str]）：

In [ ]:
def hello(name: Union[List[str], str]): 
 """Say hello to someone. 
 
    Args: 
        name (str or List[str]): name of people. 
    """ 
 print(f'hello {name} ~') 

# mypy 静态检查

- mypy 会对内置类型的表达式，做类型推导，因此内置类型不需要写 typehints 就能进行静态检查

  详见[内置类型静态检查](./typehints/mypy_builtin.py)

- mypy 可以配合 typehints，对源码中实现的类进行静态检查

  详见 [自定义类型静态检查](./typehints/mypy_custom.py)

- 基于 pre-commit hook 的 mypy 默认不会对三方库变量进行静态检查

  ![image](https://user-images.githubusercontent.com/57566630/204158317-dad10382-296f-4797-9d57-aa3864289b21.png)

  详见 [三方库变量静态检查](./typehints/mypy_3rdparty.py)





# 静态检查常见问题

> - mypy 配置项：https://mypy.readthedocs.io/en/stable/config_file.html
> - 常见问题汇总： https://mypy.readthedocs.io/en/stable/common_issues.html#variables-vs-type-aliases


## 变量的类型发生改变

```python
def parse_data(data: list): 
    data = torch.stack(data) 
    return data 
```

这段代码是无法通过 mypy 检查的。入参 data 有 Type Hints，类型为 list，而 mypy 不允许复写变量类型。因此最好的解决方案是重命名变量：

```python
# 正确 
def parse_data(data: list): 
    batch_data = torch.stack(data) 
    return batch_data
```

此外，我们也不应该复写 mypy 推导得到的变量类型：

```python
def my_func(condition) -> dict: 
    result = {'success': False} 
 
    if condition: 
        result['success'] = True 
        return result 
    else: 
        result['message'] = 'error message' 
    return result 
# error: Incompatible types in assignment (expression has type "str", target has type "bool")
```

mypy 推导得到的 results 类型为 Dict[str, bool]，而第 8 行复写为 Dict[str, str]，无法通过 mypy 检查。解决方案是预定义变量类型 :

```python
def my_func(condition) -> dict: 
    result: Dict[str, Union[str, bool]] = {'success': False} 
 
    if condition: 
        result['success'] = True 
        return result 
    else: 
        result['message'] = 'error message' 
    return result 
```

## 入参类型不匹配

对于某些变量，例如字典。写代码的人可能知道每个 key 对应的 value 是什么类型的，就可能会写出这样的代码：

```python
from typing import Dict, Union 
 
def count_chars(string) -> Dict[str, Union[str, bool, int]]: 
    result = {}  # type: Dict[str, Union[str, bool, int]] 
 
    if not isinstance(string, str): 
        result['success'] = False 
        result['message'] = 'Inavlid argument' 
    else: 
        result['success'] = True 
        result['result'] = len(string) 
    return result 
 
def get_square(integer: int) -> int: 
    return integer * integer 
 
def validate_str(string: str) -> bool: 
    check_count = count_chars(string) 
    if not check_count['success']: 
        print(check_count['message']) 
        return False 
    str_len_square = get_square(check_count['result']) 
    return bool(str_len_square > 42) 
 
result = validate_str("Lorem ipsum") 
```

我们显然知道，运行到 `get_square(check_count['result'])` 时，check_count['result'] 的类型是 int，而 get_square 的入参类型是 int。但是 mypy 并不知道这一点，因此会报错。对于这种情况，我们通常有 2 种解决方案：

- 类型窄化

    使用 isinstance 和 assert 对 check_count["result"] 做类型限定。

    ```python
    def validate_str(string: str) -> bool: 
        check_count = count_chars(string) 
        if check_count['success'] is False: 
            print(check_count['message']) 
            return False 
        assert isinstance(check_count['result'], int) 
        str_len_square = get_square(check_count['result']) 
        return bool(str_len_square > 42) 
    ```

- 类型忽视(摆烂，慎用)

    ```python
    def validate_str(string: str) -> bool: 
        check_count = count_chars(string) 
        if check_count['success'] is False: 
            print(check_count['message']) 
            return False 
        str_len_square = get_square(check_count['result'])  # type: ignore 
        return bool(str_len_square > 42) 
    ```
